# Mapping the collisions

In [1]:
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import pandas as pd
import numpy as np
import os

In [2]:
filepath = "data/"
df = pd.read_csv(filepath + "Collisions.csv")

In [3]:
df = df.dropna(subset=['Y', 'X']).reset_index()
df['INCDATE'] = pd.to_datetime(df.INCDATE)
df.isna().sum()

index                   0
X                       0
Y                       0
OBJECTID                0
INCKEY                  0
COLDETKEY               0
REPORTNO                0
STATUS                  0
ADDRTYPE                0
INTKEY             132761
LOCATION                0
EXCEPTRSNCODE      119270
EXCEPTRSNDESC      191477
SEVERITYCODE            1
SEVERITYDESC            0
COLLISIONTYPE       21130
PERSONCOUNT             0
PEDCOUNT                0
PEDCYLCOUNT             0
VEHCOUNT                0
INJURIES                0
SERIOUSINJURIES         0
FATALITIES              0
INCDATE                 0
INCDTTM                 0
JUNCTIONTYPE         7285
SDOT_COLCODE            1
SDOT_COLDESC            1
INATTENTIONIND     170683
UNDERINFL           21028
WEATHER             21189
ROADCOND            21138
LIGHTCOND           21281
PEDROWNOTGRNT      193162
SDOTCOLNUM          73555
SPEEDING           188930
ST_COLCODE           8762
ST_COLDESC          21130
SEGLANEKEY  

In [4]:
# neighborhoods = os.path.join('','seattle-neighborhoods.geojson')
# neighborhoods

In [5]:
locations = df[['Y', 'X']]
location_list = locations.values.tolist()
print(len(location_list))

197974


In [6]:
# marker clusters
m = folium.Map(location=[47.608013, -122.335167], zoom_start=11)

# marker_cluster = MarkerCluster().add_to(m)
marker_cluster = FastMarkerCluster(data=location_list).add_to(m)
folium.TileLayer('cartodbpositron').add_to(m)

folium.LayerControl().add_to(m)

for point in range(0, len(location_list)):
    folium.Marker(location_list[point], popup=str(df.INCDATE[point].date())).add_to(marker_cluster)

In [ ]:
m

In [ ]:
import subprocess
import webbrowser
from http.server import BaseHTTPRequestHandler, HTTPServer


PORT = 7000
HOST = '127.0.0.1'
SERVER_ADDRESS = '{host}:{port}'.format(host=HOST, port=PORT)
FULL_SERVER_ADDRESS = 'http://' + SERVER_ADDRESS


def TemproraryHttpServer(page_content_type, raw_data):
    """
    A simpe, temprorary http web server on the pure Python 3.
    It has features for processing pages with a XML or HTML content.
    """

    class HTTPServerRequestHandler(BaseHTTPRequestHandler):
        """
        An handler of request for the server, hosting XML-pages.
        """

        def do_GET(self):
            """Handle GET requests"""

            # response from page
            self.send_response(200)

            # set up headers for pages
            content_type = 'text/{0}'.format(page_content_type)
            self.send_header('Content-type', content_type)
            self.end_headers()

            # writing data on a page
            self.wfile.write(bytes(raw_data, encoding='utf'))

            return

    if page_content_type not in ['html', 'xml']:
        raise ValueError('This server can serve only HTML or XML pages.')

    page_content_type = page_content_type

    # kill a process, hosted on a localhost:PORT
    subprocess.call(['fuser', '-k', '{0}/tcp'.format(PORT)])

    # Started creating a temprorary http server.
    httpd = HTTPServer((HOST, PORT), HTTPServerRequestHandler)

    # run a temprorary http server
    httpd.serve_forever()


def run_html_server(html_data=None):

    if html_data is None:
        html_data = """
        <!DOCTYPE html>
        <html>
        <head>
        <title>Page Title</title>
        </head>
        <body>
        <h1>This is a Heading</h1>
        <p>This is a paragraph.</p>
        </body>
        </html>
        """

    # open in a browser URL and see a result
    webbrowser.open(FULL_SERVER_ADDRESS)

    # run server
    TemproraryHttpServer('html', html_data)

# ------------------------------------------------------------------------------------------------


# now let's save the visualization into the temp file and render it
from tempfile import NamedTemporaryFile
tmp = NamedTemporaryFile()
m.save(tmp.name)
with open(tmp.name) as f:
    folium_map_html = f.read()

run_html_server(folium_map_html)